In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

from sklearn.model_selection import GridSearchCV

from datetime import datetime

pd.set_option('display.max_rows', 5000)

zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


zri = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

#zri=zri[zri['year-month']>'2014-02']

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(6).sum()
droplist=list(zri[(zri['gapsize']==0) & (zri['year-month']>'2014-06')]['RegionName'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionName'].drop_duplicates())

0.30037614185921546

In [2]:
zhi_df2.shape

(1861, 120)

In [3]:
date_mapping={i:j for i,j in enumerate(zhi_df2.columns[6:])}

In [4]:
zri2 = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())]=zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())].fillna(method='ffill')

zri2=zri2.groupby('RegionName').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]

#zri2['ZRI']=(zri2['ZRI']/zri2['ZRI'].shift(1))-1

zri2=zri2[(zri2['year-month']>='2014-01') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionName'].isin(droplist))]

zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')

zri3=zri3.drop('RegionID', axis=1)

<ipython-input-4-5d0b3fd37d75>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')


In [5]:
#zip codes to drop
len(droplist)

559

In [6]:
#remaining filterd ZRI data:
len(zri3['RegionName'].drop_duplicates())

1302

In [7]:
zri3.isna().sum()

RegionName     0
City           0
State          0
Metro         71
CountyName     0
year-month     0
ZRI            0
dtype: int64

<H1>Train test split

In [8]:
train=zri3[(zri3['year-month']<='2018-12')]

#Y_train, Y_test=train['ZRI_scaled'], test[['ZRI_scaled']]
#X_train, X_test=train.drop('ZRI_scaled',axis=1), test.drop('ZRI_scaled',axis=1)

<H1>Change ZRI to trend only

In [9]:
for zipcode in train['RegionName'].unique():
    zip1=train[train['RegionName']==zipcode]
    zip1['year-month']=pd.to_datetime(zip1['year-month'])
    zip1=zip1.set_index('year-month')
    
    decomposed=seasonal_decompose(zip1['ZRI'],model='additive')

    trend=decomposed.trend
    seasonal=decomposed.seasonal
    residual=decomposed.resid
    
    train.loc[zri3['RegionName']==zipcode,'ZRI']=list(trend)

    #px.line(zip1['ZRI'])
    #px.line(trend)
    #px.line(seasonal)
    #px.line(residual)

<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

...

<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip1['year-month']=pd.to_datetime(zip1['year-month'])
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<ipython-input-9-4483e976490f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [10]:
train=train[(train['year-month']>'2014-06') & (train['year-month']<'2018-07')]

<H1>Standardization

In [11]:
train['ZRI_scaled']=train['ZRI']
zri3['ZRI_scaled']=zri3['ZRI']

<H1>Lag development

In [12]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [13]:
def laggenerator_diff(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+1}']=0
    return df

In [14]:
def laggenerator_diff6(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+6}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+6)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+6}']=0
    return df

In [15]:
def laggenerator_MA(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+4]
    
    df.loc[:,f'{colname}_MA{i}']=df.loc[:,f'{colname}'].shift(i)-(df.loc[:,f'{colname}'].shift(i)+df.loc[:,f'{colname}'].shift(i+1)+df.loc[:,f'{colname}'].shift(i+2)+df.loc[:,f'{colname}'].shift(i+3)+df.loc[:,f'{colname}'].shift(i+4))/5
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_MA{i}']=0
    return df

In [16]:
def laggenerator_mean(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_mean{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_mean{i}']+=df.loc[:,f'{colname}'].shift(x)
    df.loc[:,f'{colname}_mean{i}']=df.loc[:,f'{colname}_mean{i}']/i
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_mean{i}']=0
    return df

In [17]:
def laggenerator_ewm(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_ewm{i}']=0
    weightsum=0
    for y in range(1,i):
        weight=i-y
        weightsum+=weight
    for x in range(1,i):
        weight=i-x
        df.loc[:,f'{colname}_ewm{i}']+=df.loc[:,f'{colname}'].shift(x)*weight/weightsum
    df.loc[:,f'{colname}_ewm{i}']=df.loc[:,f'{colname}_ewm{i}']
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_ewm{i}']=0
    return df

In [18]:
def laggenerator_min(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_min{i}']=999999
    for x in range(1,i):
        df.loc[:,f'{colname}_min{i}']=np.minimum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_min{i}'])
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_min{i}']=0
    return df

In [19]:
def laggenerator_max(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_max{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_max{i}']=np.maximum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_max{i}'])
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_max{i}']=0
    return df

In [20]:
def lag_generator_all(df):
    for i in range(1,14):
        df=laggenerator(i, 'ZRI_scaled',df)

    for i in range(3,6):
        df=laggenerator_diff(i, 'ZRI_scaled',df)

    for i in range(1,6):
        df=laggenerator_diff6(i, 'ZRI_scaled',df)

    for i in range(3,11):
        df=laggenerator_MA(i, 'ZRI_scaled',df)

    for i in range(3,12):
        df=laggenerator_mean(i, 'ZRI_scaled',df)

    for i in range(2,12):
        df=laggenerator_ewm(i, 'ZRI_scaled',df)

    for i in range(3,14):
        df=laggenerator_min(i, 'ZRI_scaled',df)

    for i in range(3,14):
        df=laggenerator_max(i, 'ZRI_scaled',df)
    
    return df

In [21]:
train=lag_generator_all(train)
zri3=lag_generator_all(zri3)

In [22]:
train=train[(train['year-month']>'2015-06') & (train['year-month']<'2018-07')]
X_train=train.drop('ZRI_scaled',axis=1)
Y_train=train['ZRI_scaled']


In [23]:
labelencoder = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()

#X_train['StateEnc'] = labelencoder.fit_transform(X_train['State'])
X_train['CountyNameEnc'] = labelencoder2.fit_transform(X_train['CountyName'])
#X_train['CityEnc'] = labelencoder3.fit_transform(X_train['City'])
#X_train['MetroEnc'] = labelencoder4.fit_transform(X_train['Metro'])

X_train=X_train.drop(['CountyName','State','City','Metro','RegionName','year-month','ZRI'],axis=1)


In [24]:
# cat=CatBoostRegressor(logging_level='Silent',loss_function='RMSE')

# params=({
#     'random_state':[0], 
#     'n_estimators':[500], 
#     'learning_rate':[0.09,0.1,0.12], 
#     'subsample':[0.8], 
#     'max_depth':[8]
# })

# grid=GridSearchCV(cat, param_grid=params, cv=5)
# grid.fit(X_train, Y_train)
#
#grid.best_params_
#
# Result:
#     {'learning_rate': 0.09,
#  'max_depth': 8,
#  'n_estimators': 500,
#  'random_state': 0,
#  'subsample': 0.8}

In [25]:
cat=CatBoostRegressor(logging_level='Silent',loss_function='RMSE', random_state=0, n_estimators=500, learning_rate=0.09, max_depth=8, subsample=0.8)
cat.fit(X_train, Y_train)

In [26]:
cat.score(X_train, Y_train)

0.99991352167278

In [27]:

predictor_table=zri3.copy()
X_test=predictor_table[predictor_table['year-month']>'2018-12']

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    
    #run prediction for one month
    X_test=X_test[X_test['year-month']==month]
    
    #X_test['StateEnc'] = labelencoder.transform(X_test['State'])
    X_test['CountyNameEnc'] = labelencoder2.transform(X_test['CountyName'])
    #X_test['CityEnc'] = labelencoder3.transform(X_test['City'])
    #X_test['MetroEnc'] = labelencoder4.transform(X_test['Metro'])
    
    X_test=X_test.drop(['CountyName','State','City','Metro','year-month','ZRI_scaled','RegionName'],axis=1)
    val=cat.predict(X_test)

    #write current month prediction into predictor_table

    predictor_table.loc[predictor_table['year-month']==month,'ZRI_scaled']=val
        
    predictor_table=lag_generator_all(predictor_table)
        
    X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [28]:
Y_test=zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<'2020-01')]

In [29]:
#Y_test['ZRI_Yest_scaled']=Y_test['ZRI_scaled']
rstable=X_test[['ZRI_scaled','RegionName','year-month']].merge(Y_test[['ZRI_scaled']], left_index=True, right_index=True)


In [30]:
#rstable['mean']=rstable['RegionName'].apply(lambda x: std_dict[x][0])
#rstable['std']=rstable['RegionName'].apply(lambda x: std_dict[x][1])

#rstable['ZRI_scaled_x']=(rstable['ZRI_scaled_x']*rstable['std'])+rstable['mean']
#rstable['ZRI_scaled_y']=(rstable['ZRI_scaled_y']*rstable['std'])+rstable['mean']

#rstable=rstable.drop(['mean','std'],axis=1)


In [31]:
RSME=(sum((rstable['ZRI_scaled_x']-rstable['ZRI_scaled_y'])**2)/len(rstable))**0.5
RSME

104.95835019275604

In [32]:
#0.9821919015514778
#65.41642741589266

In [33]:
#zri3['mean']=zri3['RegionName'].apply(lambda x: std_dict[x][0])
#zri3['std']=zri3['RegionName'].apply(lambda x: std_dict[x][1])

#zri3['ZRI_scaled']=(zri3['ZRI_scaled']*zri3['std'])+zri3['mean']

#zri3=zri3.drop(['mean','std'],axis=1)

In [34]:
testmonths=list(zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]['year-month'].drop_duplicates())

R2_test=zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')][['ZRI_scaled','year-month','RegionName']]
R2_test['ZRI_result']=0

for i,month in enumerate(testmonths):
    R2_test.loc[R2_test['year-month']==month,'ZRI_result']=list(zri3.loc[zri3['year-month']=='2018-12','ZRI_scaled'])

RSME_null=(sum((R2_test['ZRI_scaled']-R2_test['ZRI_result'])**2)/len(R2_test))**0.5
RSME_null


63.278581823426315

In [35]:
R2=1-sum((rstable['ZRI_scaled_x']-rstable['ZRI_scaled_y'])**2)/sum((rstable['ZRI_scaled_x']-Y_test.mean()[0])**2)
R2

0.9999937706758635

In [40]:
zips=rstable['RegionName'].unique()[180:184]

In [41]:
px.line(rstable[rstable['RegionName'].isin(zips)], y=['ZRI_scaled_x','ZRI_scaled_y'],x='year-month', color='RegionName')